In [19]:
using CSV, DataFrames, JuMP, Gurobi, Formatting, NPZ

# Loading Data

In [20]:
length_s2d_df = CSV.read("csv/l2_s2d.csv", DataFrame, header=false);

In [21]:
first(length_s2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.552985,4.68216,2.8836,1.9662,1.45134,1.92868,2.31551,2.08875,1.86353
2,3.5501,1.27105,0.559142,1.91597,2.56827,1.60585,1.18909,1.33505,1.5607


In [22]:
n_src = nrow(length_s2d_df);
n_dst = ncol(length_s2d_df);

all_src = 1:n_src;
all_dst = 1:n_dst;

println("all_src: $all_src, all_dst: $all_dst")

all_src: 1:3, all_dst: 1:9


In [23]:
L = Matrix(length_s2d_df);

println("L: $(size(L))")

L: (3, 9)


# Problem
- We now additionally consider that each robot has a fixed starting source.

In [24]:
S = [5, 1, 3];
@assert size(S) == (n_src,)
@assert sum(S) == n_dst

# Minimum Distance

In [25]:
model2 = Model(Gurobi.Optimizer)

@variable(model2, x[all_src, all_dst] >= 0);

@objective(model2, Min, sum(sum(L[ii, jj] * x[ii, jj] for ii in all_src) for jj in all_dst));

# Each destination must be served by at least one source.
@constraint(model2, demand[jj in all_dst], sum(x[ii, jj] for ii in all_src) >= 1);

# There is one robot for each destination.
@constraint(model2, nrobots, sum(sum(x[ii, jj] for ii in all_src) for jj in all_dst) == n_dst);

# Each robot must start from its assigned source.
@constraint(model2, start[ii in all_src], sum(x[ii, jj] for jj in all_dst) == S[ii]);

# Solve.
optimize!(model2);
solution_summary(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 3950X 16-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 13 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x189a005d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 27 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2577251e+00   6.500000e+00   0.000000e+00      0s
      11    1.1592963e+01   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.159296336e+01

User-callback calls 69, time in user-callback 0.0

* Solver : Gurobi

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.15930e+01
  Objective bound    : 1.15930e+01
  Dual objective value : 1.15930e+01

* Work counters
  Solve time (sec)   : 2.40088e-04
  Barrier iterations : 0
  Node count         : 0


In [26]:
# Save solution.
x2 = Array(value.(x));
npzwrite("sols/problem2_l2dist.npz", Dict("x" => x2, "objective" => objective_value(model2)));

x2

3×9 Matrix{Float64}:
 1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0